<a href="https://colab.research.google.com/github/EduardoVitorInocencio/cleaning-data-with-python/blob/main/tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar as bibliotecas

import pandas                   as pd
import os
import datetime                 as dt
import re
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# Caminhos para arquivos fonte e repositório para salvar os arquivos resultados
dir_path = '/content/drive/MyDrive/Knauf/dados-knauf/'
pathDestination = '/content/drive/MyDrive/Knauf/dados-knauf-limpos/'

##########################################################################################################################
# MINHAS FUNÇÕES

def importarTratarFile (diretorio, country):

    #Diretorio contendo osm arquivos que serão combinados
    diretorio = dir_path
    country = countryFiles
    fullPath = diretorio + country + '/'

    # lista vazia para criar um repositório com os nomes dos arquivos
    res = []

    # Listar os arquivos localizados no diretórios e verificar se são arquivos em EXCEL (.xlsx)
    # caso seja, o mesmo será adicionado à lista res para agrupar os documentos na sequência

    for file in os.listdir(fullPath):

        if os.path.isfile(fullPath + file) and file.endswith('.xlsx'):
            res.append(fullPath + file)


    # Criar um data_frame vazio para agregar todas as bases localizadas no diretório anterior

    df_1 = pd.DataFrame()
    filesCombined = pd.DataFrame()

    for fullPathFile in res:
        df_1 = pd.read_excel(fullPathFile, sheet_name = 'Data')
        filesCombined = pd.concat([filesCombined,df_1])

    return filesCombined


# Função para classificar a marca de acordo com o país

def brandByCountry(row):

    if row['countryOfOrigin'] == 'CHINA':
        return 'CHINESE'
    else:
        return row['product brand']

############################################################################################################
countries = ['Argentina','Chile','Colombia','Costa Rica','Ecuador','Mexico','Panama','Paraguay','Peru']


for countryFiles in countries:
  print(countryFiles)
  # Chamada da função importarTratarFile
  files_combined = importarTratarFile(dir_path,countryFiles)


  ######################################  ARGENTINA ##########################################################

  if countryFiles == 'Argentina':

    files_combined = files_combined.loc[: ,['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Brand',
                                            'Product Schedule B Code','Product Description by Schedule B Code','Product Description','TOTAL Quantity 1',
                                            'Measure Unit 1 (Quantity-1)','TOTAL FOB Value (US$)', 'FOB Value per Unit (US$)']]

    # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)

    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','product brand','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit']

    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']

    df_filtered = pd.DataFrame()

    for brand in brands:
        df = files_combined[files_combined['product brand'].str.contains(brand) == True]
        df_filtered = pd.concat([df_filtered,df])


    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    files_combined['comercialDescription'] = files_combined['comercialDescription'].fillna('N/A')
    files_combined[files_combined['comercialDescription'].isnull()]

    for description in descriptions:

        files_combined[files_combined['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,files_combined])

    df_final = pd.concat([df_final,df_china])

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    df_final.drop(columns='product brand', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx',index=False)



     ######################################  COLÔMBIA  ##########################################################

  elif countryFiles == 'Colombia':

    files_combined.insert(0,'Product Brand',np.nan)

    files_combined = files_combined.loc[:, ['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Brand',
                                            'Product Schedule B Code','Product Description by Schedule B Code','Product Description','TOTAL Quantity 1',
                                            'Measure Unit 1 (Quantity-1)','TOTAL FOB Value (US$)','FOB Value per Unit (US$)','Supplier']]

    # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)

    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','product brand','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit','supplier']

    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']

    df_filtered = pd.DataFrame()

    for brand in brands:
        df = files_combined[(files_combined['importador'].str.contains(brand) == True) | (files_combined['supplier'].str.contains(brand) == True) ]
        df['product brand'] = brand
        df_filtered = pd.concat([df_filtered,df])


    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    df_filtered ['comercialDescription'] = df_filtered ['comercialDescription'].fillna('N/A')
    df_filtered [df_filtered['comercialDescription'].isnull()]

    for description in descriptions:

        df_filtered[df_filtered['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,df_filtered])

    df_final = pd.concat([df_final,df_china])

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    df_final.drop(columns='product brand', axis='columns',inplace = True)
    df_final.drop(columns='supplier', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx',index=False)


     ######################################  COSTA RICA  ##########################################################

  elif countryFiles == 'Costa Rica':

    files_combined.insert(5,'product brand', 'UNKNOWN')

    files_combined = files_combined.loc[:, ['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','product brand',
                                            'Product Schedule B Code','Product Description by Schedule B Code','Product Description','TOTAL Quantity 1',
                                            'Measure Unit 1 (Quantity-1)','Invoice Total Value (US$)']]

    files_combined['FOB Value per Unit (US$)'] = files_combined['Invoice Total Value (US$)'] / files_combined['TOTAL Quantity 1']


    # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)

    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','product brand','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit']

    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']

    df_filtered = pd.DataFrame()

    for brand in brands:
        df = files_combined[files_combined['product brand'].str.contains(brand) == True]
        df_filtered = pd.concat([df_filtered,df])


    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    files_combined['comercialDescription'] = files_combined['comercialDescription'].fillna('N/A')
    files_combined[files_combined['comercialDescription'].isnull()]

    for description in descriptions:

        files_combined[files_combined['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,files_combined])

    df_final = pd.concat([df_final,df_china])

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    df_final.drop(columns='product brand', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx',index=False)


     ######################################  MEXICO  ##########################################################

  elif countryFiles == 'Mexico':

    files_combined['Company ID Number'] = np.nan
    files_combined = files_combined.loc[:,['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Schedule B Code',
                                          'Product Description by Schedule B Code','Product Description','TOTAL Quantity 1','Measure Unit 1 (Quantity-1)',
                                          'TOTAL FOB Value (US$)', 'FOB Value per Unit (US$)','Foreign Company']]

    # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)

    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit','foreignCompany']

    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']


    df_filtered = pd.DataFrame()


    for brand in brands:
        df = files_combined[(files_combined['importador'].str.contains(brand) == True) | (files_combined['foreignCompany'].str.contains(brand) == True) ]
        df.loc[:,['product brand'] ]= brand
        df_filtered = pd.concat([df_filtered,df])

    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    df_filtered['comercialDescription'] = df_filtered['comercialDescription'].fillna('N/A')


    for description in descriptions:

        df_filtered[df_filtered['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,df_filtered])


    df_final = pd.concat([df_final,df_china])

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    # df_final.drop(columns='product brand', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx', index=False)


     ######################################  CHILE  ##########################################################


  elif countryFiles == 'Chile':

    files_combined = files_combined.loc[:,['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Schedule B Code',
                                          'Product Description by Schedule B Code','Product Description','TOTAL Quantity 1','Measure Unit 1 (Quantity-1)',
                                          'TOTAL FOB Value (US$)', 'FOB Value per Unit (US$)']]

    files_combined.insert(5,'product brand','UNKNOWN')


 # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)


    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','product brand','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit']


    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']


    df_filtered = pd.DataFrame()


    for brand in brands:
        df = files_combined[(files_combined['importador'].str.contains(brand) == True) | (files_combined['comercialDescription'].str.contains(brand) == True) ]
        df.loc[:,['product brand'] ]= brand
        df_filtered = pd.concat([df_filtered,df])

    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    df_filtered['comercialDescription'] = df_filtered['comercialDescription'].fillna('N/A')


    for description in descriptions:

        df_filtered[df_filtered['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,df_filtered])


    df_final = pd.concat([df_final,df_china])

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    # df_final.drop(columns='product brand', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx', index=False)


     ######################################  PANAMA  ##########################################################


  elif countryFiles == 'Panama':

    files_combined = files_combined.loc[:,['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Schedule B Code',
                                          'Product Description by Schedule B Code','Product Description','TOTAL Quantity 1','Measure Unit 1 (Quantity-1)',
                                          'TOTAL FOB Value (US$)', 'FOB Value per Unit (US$)','Consignee Company Name']]

    files_combined.insert(5,'product brand','UNKNOWN')


 # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)


    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','product brand','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit','consigneeCompanyName']


    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']


    df_filtered = pd.DataFrame()


    for brand in brands:
        df = files_combined[(files_combined['importador'].str.contains(brand) == True) |
                            (files_combined['consigneeCompanyName'].str.contains(brand) == True) |
                            (files_combined['productName'].str.contains(brand) == True)]

        df.loc[:,['product brand'] ]= brand
        df_filtered = pd.concat([df_filtered,df])

    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    df_filtered['comercialDescription'] = df_filtered['comercialDescription'].fillna('N/A')


    for description in descriptions:

        df_filtered[df_filtered['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,df_filtered])


    df_final = pd.concat([df_final,df_china])

    df_final.drop(columns='consigneeCompanyName', axis='columns',inplace=True)

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    # df_final.drop(columns='product brand', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx', index=False)


    ######################################  PARAGUAY  ##########################################################

  elif countryFiles == 'Paraguay':

    files_combined = files_combined.loc[: ,['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Brand',
                                            'Product Schedule B Code','Product Description by Schedule B Code','Product Description','TOTAL Quantity 1',
                                            'Measure Unit 1 (Quantity-1)','TOTAL FOB Value (US$)', 'FOB Value per Unit (US$)']]

    # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)

    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','product brand','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit']

    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']

    df_filtered = pd.DataFrame()

    for brand in brands:
        df = files_combined[(files_combined['product brand'].str.contains(brand) == True)|
                            (files_combined['comercialDescription'].str.contains(brand) == True)]

        df_filtered = pd.concat([df_filtered,df])


    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    files_combined['comercialDescription'] = files_combined['comercialDescription'].fillna('N/A')
    files_combined[files_combined['comercialDescription'].isnull()]

    for description in descriptions:

        files_combined[files_combined['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,files_combined])

    df_final = pd.concat([df_final,df_china])

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    df_final.drop(columns='product brand', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx',index=False)


    ######################################  DEMAIS PAÍSES ##########################################################

  else:

    files_combined = files_combined.loc[:,['Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Schedule B Code',
                                          'Product Description by Schedule B Code','Product Description','TOTAL Quantity 1','Measure Unit 1 (Quantity-1)',
                                          'TOTAL FOB Value (US$)', 'FOB Value per Unit (US$)']]

    files_combined.insert(5,'product brand','UNKNOWN')

 # Adicionar a colunas Country na tabela
    files_combined.insert(0,'country',countryFiles)


    colunas = ['country','ano','date', 'rucImportador','importador','countryOfOrigin','product brand','sku','productName','comercialDescription','quantity',
               'unidadeDeMedida','totalFoB','fobUnit']


    files_combined.columns = colunas

    df_china = files_combined.query("countryOfOrigin in ['CHINA']")

    # Gerando uma consulta dos países foco para análise e definindo quais marcas serão consideradas

    files_combined = files_combined.query("countryOfOrigin not in ['CHINA']")

    brands = ['ARMSTRONG','OWA','AMF','KNAUF AMF','KNAUF CEILING SOLUTIONS','USG','CERTAINTEED','ROCKFON','ECOPHON','HUNTER DOUGLAS','KNAUF']


    df_filtered = pd.DataFrame()


    for brand in brands:
        df = files_combined[(files_combined['importador'].str.contains(brand) == True) | (files_combined['comercialDescription'].str.contains(brand) == True) ]
        df.loc[:,['product brand'] ]= brand
        df_filtered = pd.concat([df_filtered,df])

    # Definindo as palavras-chaves que serão utilizadas para filtrar a coluna descrição, após isso é gerado um loop para identificar quais palavras
    # estão na base e após isso removemos as duplicadas, caso exista.

    descriptions = ['CIELO RASO','TECHO','ACÚSTICO','TABLEROS DE FIBRA MINERAL','BALSOSA','PLACHA','PLACA','CIELORRASO','PANEL PARA TECHO',
        'CIELO FALSO','TECHO DE FIBRA MINERAL','MINERAL FIBER CEILING','PANEL ACUSTICO','TABLEROS DE PARA CIELO RASO','PLACAS DE','AMERICANO',
        'PLANCHA DE','PAINELES','CIELOS ACUSTICOS','AISLANTE DE','CIELO RAZO']

    df_final = pd.DataFrame()

    df_filtered['comercialDescription'] = df_filtered['comercialDescription'].fillna('N/A')


    for description in descriptions:

        df_filtered[df_filtered['comercialDescription'].str.contains(description)]
        df_final = pd.concat([df_final,df_filtered])


    df_final = pd.concat([df_final,df_china])

    df_final = df_final.drop_duplicates().reset_index()

    # Classificar as marcas conforme o país, considerando a regra para País de Origem = CHINA
    df_final = df_final.assign(brand = df_final.apply(brandByCountry, axis=1))
    # df_final.drop(columns='product brand', axis='columns',inplace = True)

    # Gerar um documento final com os dados limpos e filtrados
    df_final.to_excel(pathDestination + countryFiles + '.xlsx', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Argentina
Chile


<ipython-input-23-047cf3b340c4>:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Colombia


<ipython-input-23-047cf3b340c4>:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product brand'] = brand
<ipython-input-23-047cf3b340c4>:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product brand'] = brand
<ipython-input-23-047cf3b340c4>:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Costa Rica
Ecuador


...

<ipython-input-23-047cf3b340c4>:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Panama


<ipython-input-23-047cf3b340c4>:430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Paraguay
Peru


<ipython-input-23-047cf3b340c4>:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['product brand'] ]= brand
<ipython-input-23-047cf3b340c4>:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [ ]:
files_combined.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8500 entries, 0 to 4249
Data columns (total 14 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   country                                 8500 non-null   object 
 1   Year                                    8500 non-null   int64  
 2   Date yyyy-mm-dd                         8500 non-null   object 
 3   Company ID Number                       8500 non-null   int64  
 4   Company declarant                       8500 non-null   object 
 5   Country of Origin                       8500 non-null   object 
 6   product brand                           8500 non-null   object 
 7   Product Schedule B Code                 8500 non-null   int64  
 8   Product Description by Schedule B Code  8500 non-null   object 
 9   Product Description                     8500 non-null   object 
 10  TOTAL Quantity 1                        8500 non-null   floa

In [ ]:
  elif countryFiles == 'Chile':

    files_combined = files_combined.loc[:,
                                          [
                                            'Year','Date yyyy-mm-dd','Company ID Number','Company declarant','Country of Origin','Product Schedule B Code',
                                          'Product Description by Schedule B Code','Product Description','TOTAL Quantity 1','Measure Unit 1 (Quantity-1)',
                                          'TOTAL FOB Value (US$)', 'FOB Value per Unit (US$)'
                                          ]
                                    ]
    files_combined.insert(5,'product brand','UNKNOWN')




,country,ano,date,rucImportador,importador,countryOfOrigin,product brand,sku,productName,comercialDescription,quantity,unidadeDeMedida,totalFoB,fobUnit
